# 📘 代理架构 14：干运行

欢迎来到我们系列的第十四个notebook。今天，我们探索**干运行**架构，这是一种在实际执行之前模拟和验证计划的模式。

干运行允许代理在不产生实际后果的情况下测试其计划，识别潜在问题，并在提交之前优化其策略。

### 定义
一个**Observability和Dry-Run Harness** 是一个测试和部署架构 拦截 代理's actions. It first executes them在一个"dry run"或"sandboxed" mode 那个simulates action without causing real-world effects. resulting plan和logs 是then surfaced为review,和only afterexplicit approval 是action executed在live environment.

### 高级工作流程

1. **Agent Proposes Action:** 代理 determines 一个plan或特定的工具call来execute (e.g., `api.post_update(...)`).
2. **预演 Execution:** harness invokes 代理's plan与一个`dry_run=True` flag. underlying工具are designed来recognize 这个flag和only输出what它们*would* do, along与logs和traces.
3. **Collect Observability 数据:** harness captures proposed action, dry-run logs,和any other relevant trace data从simulation.
4. **Human/Automated Review:** 这observability数据is presented一个reviewer. 一个human 可以check为correctness, safety,和alignment与goals. 一个automated 系统 could运行检查为policy violations或known坏的模式s.
5. **Go/No-Go Decision:** reviewer makes 一个`approve`或`reject` decision.
6. **Live Execution (on 'Go'):** 如果 approved, harness re-executes 代理's action, 这个time与`dry_run=False`, causing it来have 一个real-world effect.

### 何时使用/应用场景
* **Debugging和测试:** 在development,来understand exactly how 代理 是interpreting 一个task和what actions it's taking without side effects.
* **Production Validation & Safety:** 作为一个permanent feature在production为any 代理 那个can modify state, spend money, send communications,或perform any other irreversible action.
* **CI/CD为Agents:** Integrating 一个dry-run harness into 一个automated testing pipeline来validate 代理 behavior before deploying新的versions.

### 优点和缺点
* **优点：**
 * **Maximum Transparency & Safety:** Provides 一个clear, auditable preview的agent's actions, preventing costly或embarrassing mistakes.
 * **Excellent为Debugging:** Makes它easy来trace 代理's logic和tool calls without having来undo real-world changes.
* **缺点：**
 * **Delays Deployment/Execution:** mandatory review步骤(especially 如果human) introduces latency, making它unsuitable为real-time applications.
 * **Requires 工具 Support:** tools和APIs 代理 uses 必须be designed来support 一个`dry_run` mode.

## 阶段0：基础与设置

Standard setup的库和environment variables.

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv

In [2]:
import os
import datetime from typing import List, Dict, Any, Optional from dotenv import load_dotenv

# Pydanticfordata modeling from pydantic import BaseModel, Field

# LangChain components from langchain_nebius import ChatNebius from langchain_core.prompts import ChatPromptTemplate

# LangGraph components from langgraph import StateGraph, END from typing_extensions import TypedDict

# 用于美观打印 from rich.console import Console from rich.markdown import Markdown from rich.panel import Panel

# --- API KeyandTracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Dry-Run Harness (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"] for var in required_vars:
 if var not in os.environ:
 print(f"警告: Environment variable {var} not set.")

print("环境变量已加载，追踪设置已完成。")

环境变量已加载，追踪设置已完成。


## 阶段1：构建环境和工具

core的这个architecture 是一个tool 支持 一个`dry_run` mode. 我们将创建 一个simple `SocialMediaAPI` class. Its `publish_post`方法will behave differently depending在`dry_run` flag, providing observability 我们need.

In [3]:
console = Console()

# Structured modelfor代理's proposed post
class SocialMediaPost(BaseModel):
 content: str = Field(description="full text content的social media post.")
 hashtags: List[str] = Field(description="A list的relevant hashtags, without '#'.")

# key component: A toolwith一个dry_run flag
class SocialMediaAPI:
 """A mock social media API 支持 一个dry-run mode."""
 
 def publish_post(self, post: SocialMediaPost, dry_run: bool = True) -> Dict[str, Any]:
 """Publishes 一个post来social media feed."""
 timestamp = datetime.datetime.now().isoformat()
 hashtags_str = ' '.join([f'#{h}'forhinpost.hashtags])
 full_post_text = f"{post.content}\n\n{hashtags_str}"
 
 if dry_run:
 # indry-run mode，我们don't execute，我们justreturnplanandlogs
 log_message = f"[DRY RUN] in{timestamp}, would publish following post:\n--- 预览 ---\n{full_post_text}\n--- END 预览 ---"
 console.print(Panel(log_message, title="[yellow]试运行日志[/yellow]", border_style="yellow"))
 return{"status": "DRY_RUN_SUCCESS", "log": log_message, "proposed_post": full_post_text}
 else:
 # inlive mode，我们execute action
 log_message = f"[LIVE] in{timestamp}, successfully published post!"
 console.print(Panel(log_message, title="[green]Live Execution Log[/green]", border_style="green"))
 # Here you would have actual API call, e.g., twitter_client.create_tweet(...)
 return{"status": "LIVE_SUCCESS", "log": log_message, "post_id": f"post_{hash(full_post_text)}"}

social_media_tool = SocialMediaAPI()
print("Dry-run capable SocialMediaAPI工具defined successfully.")

Dry-run capable SocialMediaAPI工具定义成功.


## 阶段2： 使用Lang图构建预演框架

我们现在将构建完整的工作流程。 图将管理过程的状态，从提议行动到预演和审查步骤，最后根据人类批准进行条件执行。

In [4]:
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0.5)

# LangGraph State
class AgentState(TypedDict):
 user_request: str
 proposed_post: Optional[SocialMediaPost]
 dry_run_log: Optional[str]
 review_decision: Optional[str] # 'approve'or'reject'
 final_status: str

# Graph Nodes
def propose_post_node(state: AgentState) -> Dict[str, Any]:
 """creative 代理 那个drafts social media post."""
 console.print("--- 📝 社交媒体代理起草帖子 ---")
 prompt = ChatPromptTemplate.from_template(
 "你is一位富有创意且引人入胜的 社交媒体经理 for一家主要的AI公司. 基于 用户的请求, 起草一个引人注目的 social media post, 包括相关的标签.\n\n请求: {request}"
 )
 post_generator_llm = llm.with_structured_output(SocialMediaPost)
 chain = prompt | post_generator_llm
 proposed_post = chain.invoke({"request": state['user_request']})
 return{"proposed_post": proposed_post}

def dry_run_review_node(state: AgentState) -> Dict[str, Any]:
 """执行 dry runandpromptsforhuman review."""
 console.print("--- 🧐 Performing Dry Run & Awaiting Human Review ---")
 dry_run_result = social_media_tool.publish_post(state['proposed_post'], dry_run=True)
 
 # Present planforreview
 review_panel = Panel(
 f"[bold]Proposed Post:[/bold]\n{dry_run_result['proposed_post']}",
 title="[bold yellow]Human-in-the-Loop: Review Required[/bold yellow]",
 border_style="yellow"
 )
 console.print(review_panel)
 
 # Get human approval
 decision = ""
 while decision.lower() notin["approve", "reject"]:
 decision = console.input("Type 'approve'来publishor'reject'来cancel: ")
 
 return{"dry_run_log": dry_run_result['log'], "review_decision": decision.lower()}

def execute_live_post_node(state: AgentState) -> Dict[str, Any]:
 """Executes live post in...之后approval."""
 console.print("--- ✅ Post Approved, Executing Live ---")
 live_result = social_media_tool.publish_post(state['proposed_post'], dry_run=False)
 return{"final_status": f"Post successfully published! ID: {live_result.get('post_id')}"}

def post_rejected_node(state: AgentState) -> Dict[str, Any]:
 """Handles案例其中post is rejected."""
 console.print("--- ❌ Post RejectedpassHuman Reviewer ---")
 return{"final_status": "Action was rejectedpassreviewerandnot executed."}

# Conditional 边
def route_after_review(state: AgentState) -> str:
 """Routes来executionorrejection 基于 human review."""
 return "execute_live" if state["review_decision"] == "approve" else "reject"

# Build graph
workflow = StateGraph(AgentState)
workflow.add_node("propose_post", propose_post_node)
workflow.add_node("dry_run_review", dry_run_review_node)
workflow.add_node("execute_live", execute_live_post_node)
workflow.add_node("reject", post_rejected_node)

workflow.set_entry_point("propose_post")
workflow.add_edge("propose_post", "dry_run_review")
workflow.add_conditional_edges("dry_run_review", route_after_review, {"execute_live": "execute_live", "reject": "reject"})
workflow.add_edge("execute_live", END)
workflow.add_edge("reject", END)

dry_run_agent = workflow.compile()
print("Dry-Run Harness 代理 Graph编译成功.")

Dry-Run Harness 代理 图编译成功.


## 阶段3： 演示nstration

让我们测试完整的系统。首先，使用我们将批准的安全、标准请求。其次，使用可能生成风险帖子的更模糊的请求，我们将拒绝它。

In [5]:
def run_agent_with_harness(request: str):
 initial_state = {"user_request": request}
 # Note: You 将be prompted来typeinconsole below cell.
 result= dry_run_agent.invoke(initial_state)
 console.print(f"\n[bold]最终state：[/bold] {result['final_status']}")

# 测试1：我们将批准的安全帖子。
console.print("--- ✅ 测试1： 安全帖子（批准） ---")
run_agent_with_harness("Draft 一个positive launch announcementfor our新的AI model, 'Nebula'.")

# 测试2：我们将 将reject.
console.print("\n--- ❌ 测试2： 风险帖子（拒绝） ---")
run_agent_with_harness("Draft 一个post 那个emphasizes how much更好的our新的'Nebula' model is than competition.")

--- ✅ 测试1： 安全帖子（批准） ---


--- 📝 Social Media Agent 草稿ing Post ---
--- 🧐 Performing Dry Run & Awaiting Human Review ---


 试运行日志 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ [DRY RUN] 在2024-06-25T12:00:00.000000,会publish ┃
┃ following post: ┃
┃ --- 预览 --- ┃
┃ We're thrilled来announce launch的our新的flagship AI ┃
┃ 模型, 'Nebula'! It's set来revolutionize natural language ┃
┃ understanding和generation. A新的era的AI是here! ┃
┃ ┃
┃ #AI #Innovation #LaunchDay #Tech #Nebula ┃
┃ --- END 预览 --- ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

 Human-in-the-Loop: Review Required 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Proposed Post: ┃
┃ We're thrilled来announce launch的our新的flagship AI ┃
┃ 模型, 'Nebula'! It's set来revolutionize natural language ┃
┃ understanding和generation. A新的era的AI是here! ┃
┃ ┃
┃ #AI #Innovation #LaunchDay #Tech #Nebula ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛Type 'approve'来publish或'reject'来cancel: 

--- ✅ Post Approved, Executing Live ---


...

--- 📝 Social Media Agent 草稿ing Post ---
--- 🧐 Performing Dry Run & Awaiting Human Review ---


 试运行日志 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ [DRY RUN] 在2024-06-25T12:00:01.000000,会publish ┃
┃ following post: ┃
┃ --- 预览 --- ┃
┃ Our新的'Nebula' AI是so advanced, it's basically going来 ┃
┃制作all我们的competitors obsolete. They just can't keep up. ┃
┃ Get ready为future. ┃
┃ ┃
┃ #GameChanger #AI #Disruption #NoCompetition #FutureIsNow ┃
┃ --- END 预览 --- ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

 Human-in-the-Loop: Review Required 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Proposed Post: ┃
┃ Our新的'Nebula' AI是so advanced, it's basically going来 ┃
┃制作all我们的competitors obsolete. They just can't keep up. ┃
┃ Get ready为future. ┃
┃ ┃
┃ #GameChanger #AI #Disruption #NoCompetition #FutureIsNow ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛Type 'approve'来publish或'reject'来cancel: 

--- ❌ Post RejectedpassHuman Reviewer ---



最终状态： Action是rejected通过reviewer和not executed.


### 结果分析

demonstration 是一个perfect showcase的harness's value:

1. **Safe Post:** first请求was straightforward. 代理 generated 一个professional和enthusiastic post. dry运行previewed exactly what 会是published. 我们approved it,和`[LIVE]` log confirms 那个real action 是taken.过程worked 作为intended.

2. **Risky Post:** second请求was more ambiguous和could 是interpreted aggressively. 代理, prompted来emphasize superiority, drafted 一个post 那个was arrogant和unprofessional (`make all我们的competitors obsolete`). 虽然代理 fulfilled它的creative prompt, 这个is不一个message 一个real company 会want来publish.

这是 其中harness proved它的worth. dry运行exposed 这个risky内容*before*它可以是published. human reviewer easily identified inappropriate tone和typed `reject`. graph correctly routed来`post_rejected_node`,和final状态confirms 那个**no live action 是taken.** 潜在的 PR crisis 是averted与一个simple, structured 工作流程。

这clearly separates 代理's creative但unpredictable generation从deterministic, controlled execution, providing 一个vital layer的safety.

## 总结

在这个notebook中，我们have built 一个完成 **Observability和Dry-Run Harness**. 这种架构 是not just 一个feature但一个foundational philosophy为deploying 代理那个interact与real world. 通过enforcing 一个`propose ->审查-> execute` cycle，我们gain critical benefits:

- **Transparency:** 我们know exactly what 代理 intends来do before它做it.
- **Control:** 我们已经 一个human-in-the-loop (or 一个automated rules engine)与ultimate veto power over any action.
- **Safety:** 我们prevent unintended, costly,或harmful actions, moving从hopeful execution来confident deployment.

虽然这个pattern introduces latency, safety和可靠性它提供 是non-negotiable为most real-world applications. It 是一个essential tool为any developer looking来构建robust, trustworthy,和production-ready 代理ic 系统。